In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install optuna
import optuna

In [3]:
import numpy as np
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import OneHotEncoder

In [4]:
path="/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/"

In [5]:
test = pd.read_json(path+'ATE_ABSITA_test_set/ate_absita_gold.ndjson',
                    lines=True)

train = pd.read_json(path+'ATE_ABSITA_training_set/ate_absita_training.ndjson',
                     lines=True)

data_sinonimi = pd.read_csv(path+"ATE_ABSITA_test_set/sinonimi.csv")
data_embedding = pd.read_csv(path+"ATE_ABSITA_test_set/embedding.csv")

In [ ]:
print(test.columns)
print(train.columns)
print(data_sinonimi.columns)
print(data_embedding.columns)

In [8]:
train.drop(columns=['id_sentence','polarities','aspects_position','aspects'], inplace=True)
test.drop(columns=['id_sentence','polarities','aspects_position','aspects'], inplace=True)
data_sinonimi.drop(columns=['polarities','aspects_position','aspects'], inplace=True)
data_embedding.drop(columns=['polarities','aspects_position','aspects'], inplace=True)
print(f'Contains {len(train)} sentences')
print(f'Contains {len(test)} sentences')
print(f'Contains {len(data_sinonimi)} sentences')
print(f'Contains {len(data_embedding)} sentences')

Contains 3054 sentences
Contains 1200 sentences
Contains 1200 sentences
Contains 1200 sentences


Creazione colonna Positivi/Negativi

In [9]:
train["review_type"] = train["score"].apply(lambda x: "neg" if x < 5 else "pos")
test["review_type"] = test["score"].apply(lambda x: "neg" if x < 5 else "pos")
data_sinonimi["review_type"] = data_sinonimi["score"].apply(lambda x: "neg" if x < 5 else "pos")
data_embedding["review_type"] = data_embedding["score"].apply(lambda x: "neg" if x < 5 else "pos")

print(f'TRAIN::\n{train.review_type.value_counts()}')
print(f'TEST::\n{test.review_type.value_counts()}')
print(f'SINONIMI::\n{data_sinonimi.review_type.value_counts()}')
print(f'EMBEDDING::\n{data_embedding.review_type.value_counts()}')

TRAIN::
pos    2150
neg     904
Name: review_type, dtype: int64
TEST::
pos    857
neg    343
Name: review_type, dtype: int64
SINONIMI::
pos    857
neg    343
Name: review_type, dtype: int64
EMBEDDING::
pos    857
neg    343
Name: review_type, dtype: int64


Rimozione Colonna Score in quanto non piu significativa per la Sentiment Analysis

In [10]:
train.drop(columns=['score'], inplace=True)
test.drop(columns=['score'], inplace=True)
data_sinonimi.drop(columns=['score'], inplace=True)
data_embedding.drop(columns=['score'], inplace=True)

In [11]:
with open(path+"word2index.pkl", 'rb') as output:
  w2i = pickle.load(output)
with open(path+"embedding_matrix.pkl", 'rb') as output:
  embedding_matrix = pickle.load(output)

Trasformazione input da frasi a vettori di parole

In [12]:
def my_text_to_word_sequence(sentence):
  return keras.preprocessing.text.text_to_word_sequence(sentence,
                                                        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`\'{|}~\t\n',
                                                        lower=True)

TRAIN, Encoding e Dataset per modello

In [13]:
one_hot_train = OneHotEncoder(sparse=False).fit_transform(
        train.review_type.to_numpy().reshape(-1, 1)
        )

In [14]:
sentences = [my_text_to_word_sequence(sentence) for sentence in train['sentence']]

TEST, Encoding e Dataset per modello

In [15]:
one_hot_test = OneHotEncoder(sparse=False).fit_transform(
        test.review_type.to_numpy().reshape(-1, 1)
        )

In [16]:
sentences_test = [my_text_to_word_sequence(sentence) for sentence in test['sentence']]

SINONIMI, Encoding e Dataset per modello

In [17]:
one_hot_sin = OneHotEncoder(sparse=False).fit_transform(
        data_sinonimi.review_type.to_numpy().reshape(-1, 1)
        )

In [18]:
sentences_sin = [my_text_to_word_sequence(sentence) for sentence in data_sinonimi['sentence']]

EMBEDDING, Encoding e Dataset per modello

In [19]:
one_hot_emb = OneHotEncoder(sparse=False).fit_transform(
        data_embedding.review_type.to_numpy().reshape(-1, 1)
        )

In [20]:
sentences_emb = [my_text_to_word_sequence(sentence) for sentence in data_embedding['sentence']]

Estrai la massima dimensione dell'input in base ai vari dataset considerati

In [21]:
max_index, max = (-1, -1)
for i, sentence in enumerate(sentences):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_test):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_sin):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_emb):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)


print(f'Il massimo è {max}')

Il massimo è 85


Creo i vari embedding per tutti i dataset, quest'operazione e pesante 

In [26]:
embedded_train = np.zeros(shape=(len(sentences), max, 300))
for i, sentence in enumerate(sentences):
  for j, word in enumerate(sentence):
    try:
      embedded_train[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [27]:
embedded_test = np.zeros(shape=(len(sentences_test), max, 300))
for i, sentence in enumerate(sentences_test):
  for j, word in enumerate(sentence):
    try:
      embedded_test[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [28]:
embedded_sin = np.zeros(shape=(len(sentences_sin), max, 300))
for i, sentence in enumerate(sentences_sin):
  for j, word in enumerate(sentence):
    try:
      embedded_sin[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [29]:
embedded_emb = np.zeros(shape=(len(sentences_emb), max, 300))
for i, sentence in enumerate(sentences_emb):
  for j, word in enumerate(sentence):
    try:
      embedded_emb[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

#Model

In [30]:
best_params = optuna.load_study(study_name="ATE", storage="sqlite:///"+path+"optuna_ATE_studio_0.db").best_params

In [31]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(max, 300)))
model.add(keras.layers.Bidirectional(layer=keras.layers.LSTM(units=best_params["units"],
                                                             recurrent_dropout=best_params["dropout"],
                                                             activation='tanh')))
model.add(keras.layers.Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
            optimizer=keras.optimizers.Adam(0.001),
            metrics=['accuracy'])

In [33]:
best_params

{'batch_size': 55, 'dropout': 0.28, 'units': 125}

In [34]:
#train model

#embedded_* è il nome da cambiare per allenare il modello sul dataseta selezionato per lo studio in esame

#one_hot_* è il nome dell'encoding delle parole del dataseta selezionato per lo studio in esame
result = model.fit(embedded_sin,
                 one_hot_sin,
                 epochs=100,
                 batch_size=best_params["batch_size"],
                 callbacks=[keras.callbacks.EarlyStopping(monitor='loss',
                                                            patience=10)])

Epoch 1/100
22/22 [==============================] - 16s 448ms/step - loss: 0.6353 - accuracy: 0.6805
Epoch 2/100
22/22 [==============================] - 10s 444ms/step - loss: 0.5704 - accuracy: 0.7320
Epoch 3/100
22/22 [==============================] - 10s 446ms/step - loss: 0.5733 - accuracy: 0.7295
Epoch 4/100
22/22 [==============================] - 10s 457ms/step - loss: 0.5332 - accuracy: 0.7579
Epoch 5/100
22/22 [==============================] - 10s 448ms/step - loss: 0.5131 - accuracy: 0.7480
Epoch 6/100
22/22 [==============================] - 10s 452ms/step - loss: 0.5074 - accuracy: 0.7427
Epoch 7/100
22/22 [==============================] - 10s 454ms/step - loss: 0.4926 - accuracy: 0.7575
Epoch 8/100
22/22 [==============================] - 10s 448ms/step - loss: 0.4753 - accuracy: 0.7838
Epoch 9/100
22/22 [==============================] - 10s 454ms/step - loss: 0.4501 - accuracy: 0.7847
Epoch 10/100
22/22 [==============================] - 10s 444ms/step - loss: 0.442

In [35]:
#save the model
# cambiare il nome in base allo studio che si svolge
model.save_weights(path+'ATE_w_studio_1.h5')

In [36]:
# cambiare il nome in base allo studio che si svolge
model.load_weights(path+'ATE_w_studio_1.h5')

#EVALUATION

In [37]:
result_base=model.evaluate(embedded_trainset,one_hot_train,batch_size=best_params['batch_size'],)
print(f'DATASET ORIGINARIO{result_base}')

56/56 [==============================] - 3s 42ms/step - loss: 1.7612 - accuracy: 0.6811
DATASET ORIGINARIO[1.7612053155899048, 0.6810740232467651]


In [38]:
result_base=model.evaluate(embedded_test,one_hot_test,batch_size=best_params['batch_size'],)
print(f'DATASET DEV{result_base}')

22/22 [==============================] - 1s 51ms/step - loss: 0.3679 - accuracy: 0.9067
DATASET DEV[0.367866575717926, 0.9066666960716248]


In [39]:
result_base=model.evaluate(embedded_sin,one_hot_sin,batch_size=best_params['batch_size'])
print(f'DATASET SINONIMI{result_base}')

22/22 [==============================] - 1s 49ms/step - loss: 0.0152 - accuracy: 0.9975
DATASET SINONIMI[0.01519171055406332, 0.9975000023841858]


In [40]:
result_base=model.evaluate(embedded_emb,one_hot_emb,batch_size=best_params['batch_size'])
print(f'DATASET EMBEDDING{result_base}')

22/22 [==============================] - 1s 44ms/step - loss: 0.7684 - accuracy: 0.8025
DATASET EMBEDDING[0.7684059143066406, 0.8025000095367432]
